In [22]:
# Necessary Imports

import os, random, sys, copy
import torch, torch.nn as nn, numpy as np
from tqdm.notebook import tqdm
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from nltk.tokenize import word_tokenize


from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline
from datasets import load_dataset
from evaluate import load
from tqdm.notebook import tqdm

import random

We must now load the dataset from our local JSON files. 

For refernece, see https://huggingface.co/docs/datasets/en/loading

In [32]:
from datasets import load_dataset

#Load the dataset
#Todo: Split into train/test, right now this is all training
squad_dataset = load_dataset("json", data_files=["QQA Data/QQA_dev.json", "QQA Data/QQA_test.json", "QQA Data/QQA_train.json"])

print('Q: ' + dataset['train'][0]['question'])
print('A: ' + dataset['train'][0]['Option1'])
print('A: ' + dataset['train'][0]['Option2'])
print('A: ' + dataset['train'][0]['answer'])

#Load evaluation metric
accuracy_metric = load("squad")

Q: Jame's mother has a photo of Jane standing at a height of 14 inches, whereas a mountain appears to have height of 26 cm. It looks that way because? 
A: the mountain was farther away
A: Jane was farther away
A: Option 2


The imported data is of the form:

{"question": "Jame's mother has a photo of Jane standing at a height of 14 inches, whereas a mountain appears to have height of 26 cm. It looks that way because? ", "Option1": "the mountain was farther away", "Option2": "Jane was farther away", "answer": "Option 2", "type": "Type_3", "question_sci_10E": "Jame's mother has a photo of Jane standing at a height of 1.4000000000E+01 inches, whereas a mountain appears to have height of 2.6000000000E+01 cm. It looks that way because? ", "question_char": "Jame's mother has a photo of Jane standing at a height of 1 4 inches, whereas a mountain appears to have height of 2 6 cm. It looks that way because? ", "question_sci_10E_char": "Jame's mother has a photo of Jane standing at a height of 1 . 4 0 0 0 0 0 0 0 0 0 E + 0 1 inches, whereas a mountain appears to have height of 2 . 6 0 0 0 0 0 0 0 0 0 E + 0 1 cm. It looks that way because? ", "question_mask": "Jame's mother has a photo of Jane standing at a height of [Num] inches, whereas a mountain appears to have height of [Num] cm. It looks that way because? "

For reference, see:
https://huggingface.co/docs/transformers/en/tasks/multiple_choice


In [30]:
#Todo

model_name = 'deepset/roberta-base-squad2'

def evaluate_hf_model(model_name):
    model = AutoModelForQuestionAnswering.from_pretrained(model_name)       # Initialize the model
    tokenizer = AutoTokenizer.from_pretrained(model_name)                   # Initialize the tokenizer

    processor = pipeline('question-answering', model=model, tokenizer=tokenizer)

    def dataset_generator(dataset):
        for ex in dataset:
            yield (ex,
                {'question' : ex['question'], 'context': ex['context']})
            
    predictions = []
    references = []

    # Get predictions, and save corresponding reference (if we were using the whole dataset, we wouldn't need this step)
    for ex in tqdm(dataset_generator(squad_dataset), total=len(squad_dataset)):

        predictions.append({
                'id' : ex[0]['id'],
                'prediction_text' : processor(ex[1])['answer']
        }
        )

        # In each example, there are multiple possible answers which we compare to. Here we are converting from them from the datasets format to the one expected by the evaluation metric. 
        references.append({
            'id' : ex[0]['id'],
            'answers' : [{'text' : z[0], 'answer_start' : z[1]} for z in zip(ex[0]['answers']['text'], ex[0]['answers']['answer_start'])]
        })

    # Compute metrics
    print('Performance of {} : {}'.format(model_name, squad_evaluate.compute(predictions=predictions, references=references)))

evaluate_hf_model(model_name)

  0%|          | 0/1 [00:00<?, ?it/s]

TypeError: string indices must be integers

For evaluation, see https://huggingface.co/spaces/evaluate-metric/accuracy

For fine-tuning, see: https://huggingface.co/docs/transformers/en/training#train-with-pytorch-trainer

For handing multiple choice, see https://huggingface.co/docs/transformers/en/tasks/multiple_choice